# CHSH不等式の破れを確認する

$\newcommand{\ket}[1]{|#1\rangle}$

## 必要なモジュールのインポート

In [ ]:
# まずは必要になるpythonモジュールをすべてインポートしておく
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, IBMQ, Aer, execute
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram

print('notebook ready')

## 初めての量子回路

In [ ]:
circuit = QuantumCircuit(2) # レジスタを介さずビット数を指定して回路を作成することもできます
circuit.h(0) # その場合、ゲートにはregister[0]ではなく直接量子ビットの番号を指定します
circuit.ry(np.pi / 2., 0) #　θ = π/2
circuit.x(0)
circuit.measure_all()

print('This circuit has', circuit.num_qubits, 'qubits and', circuit.size(), 'operations')

## 制御ゲートの使用

制御ゲートを使って状態

$$
\sqrt{\frac{1}{10}} \ket{00} + \sqrt{\frac{2}{10}} \ket{01} + \sqrt{\frac{3}{10}} \ket{10} - \sqrt{\frac{4}{10}} \ket{11}
$$

を作ります。

In [ ]:
theta1 = 2. * np.arctan(np.sqrt(7. / 3.))
theta2 = 2. * np.arctan(np.sqrt(2.))
theta3 = 2. * np.arctan(np.sqrt(4. / 3))

circuit = QuantumCircuit(2)
circuit.ry(theta1, 1)
circuit.ry(theta2, 0)
circuit.cry(theta3 - theta2, 1, 0) # C[Ry]　1が制御で0が標的
circuit.cz(0, 1) # C[Z] 0が制御で1が標的（実はC[Z]ではどちらが制御でも結果は同じ）
circuit.measure_all()

print('This circuit has', circuit.num_qubits, 'qubits and', circuit.size(), 'operations')

## 回路図

In [ ]:
circuit.draw('mpl')

In [ ]:
circuit.draw()

## CHSH不等式を計算する回路を書く

4つの回路でそれぞれ「ベル状態」

$$
\frac{1}{\sqrt{2}} \left( \ket{00} + \ket{11} \right)
$$

を作ったあと、別々の$R_y$ゲートをかけます。

In [ ]:
circuits = []
for ic in range(4):
    circuit = QuantumCircuit(2, name='circuit{}'.format(ic))
    circuit.h(0)
    circuit.cx(0, 1)
    circuits.append(circuit)

circuits[0].ry(-np.pi / 4., 1)
circuits[1].ry(-3. * np.pi / 4., 1)
circuits[2].ry(-np.pi / 4., 1)
circuits[3].ry(-3. * np.pi / 4., 1)

circuits[2].ry(-np.pi / 2., 0)
circuits[3].ry(-np.pi / 2., 0)

for circuit in circuits:
    circuit.measure_all()

# draw()にmatplotlibのaxesオブジェクトを渡すと、そこに描画してくれる
# 一つのノートブックセルで複数プロットしたい時などに便利
for circuit in circuits:
    ax = plt.figure().add_subplot()
    circuit.draw('mpl', ax=ax)

## 回路をシミュレータで実行する

ワークブックでは直接回路を実機に渡していますが、講義では時間が限られているので、まずシミュレータを利用します。

### ジョブの実行

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
print(simulator.name())

shots = 10000
    
sim_job = execute(circuits, backend=simulator, shots=shots)

sim_result = sim_job.result()

### 測定結果の解析 - 個々の回路のヒストグラム

In [ ]:
sim_counts = []
for circuit in circuits:
    c = sim_result.get_counts(circuit)
    sim_counts.append(c)
    
for c in sim_counts:
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)

### 測定結果の解析 - CHSH不等式の$S$

In [ ]:
C = []
for c in sim_counts:
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]
print('S =', S)

## 回路を実機で実行する

### IBMQ認証

In [ ]:
IBMQ.enable_account('__paste_your_token_here__')

### バックエンドの決定

In [ ]:
# IBMQプロバイダ（実機へのアクセスを管理するオブジェクト）
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# バックエンド（実機）のうち量子ビット数2個以上のもののリストをプロバイダから取得し、一番空いているものを選ぶ
backend_filter = lambda b: (not b.configuration().simulator) and (b.configuration().n_qubits >= 2) and b.status().operational
backend = least_busy(provider.backends(filters=backend_filter))

print('Jobs will run on', backend.name())

### ジョブの実行

In [ ]:
shots = 8192

job = execute(circuits, backend=backend, shots=shots)

job_monitor(job, interval=2)

result = job.result()

### ヒストグラム

In [ ]:
counts = []
for circuit in circuits:
    c = result.get_counts(circuit)
    counts.append(c)
    
for c in counts:
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)

In [ ]:
C = []
for c in counts:
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]

print('C:', C)
print('S =', S)
if S > 2.:
    print('Yes, we are using a quantum computer!')